# Predicting Horse Racing Results
BA Report (August 2020) <br>
Andrew Woon
## 1. Data Acquisition 

#### Required libraries
> scrapy <br>
> scrapy_splash <br>
> scrapy_fake_useragent <br>
> tabula <br>

#### Required Software
> Docker

### 1.1 Data Scraping

Scrapy was used as the primary data scraping library. However, as the Hong Kong Jockey Club horse racing results page (https://racing.hkjc.com/racing/information/English/racing/LocalResults.aspx) is loaded via JavaScript, the core scrapy library is insufficient and the Scrapy Splash extension was therefore needed. 

The Scrapy Splash library requires Docker to be installed as it uses a docker container to load the JavaScript websites and return a HTML output that scrapy is able to operate on. 
Scrapy Splash user guide reference: https://github.com/scrapy-plugins/scrapy-splash

<b>File Directory </b>
```
└── hkjc
    ├── hkjc
    │   ├── __pycache__
    │   ├── spiders
    │   │   ├── __init__.py
    │   │   └── hkjc_spider.py
    │   ├── items.py
    │   ├── middlewares.py
    │   ├── overseas_pdfparser.py
    │   ├── overseas_rerun.py
    │   ├── pipelines.py
    │   └── settings.py
    ├── hkjc_racing_fixtures.csv            #input file
    ├── dividend_YYYYMMDDhhmm.csv           #output file format, example dividend_202008182011.csv
    ├── horseform_YYYYMMDDhhmm.csv          #output file format
    ├── horseinfo_YYYYMMDDhhmm.csv          #output file format
    ├── race_YYYYMMDDhhmm.csv               #output file format
    └── sectional_YYYYMMDDhhmm.csv          #output file format
```

Within the `./hkjc/hkjc` folder all the python files except for `overseas_pdf_parser.py` and `overseas_rerun.py` are the default scrapy required files (customised for this use case)

Scrapy uses 'spiders' to crawl unique webpages and this core logic is contained in the `spider.py` file. For this use case there are 3 unique pages to crawl:

| # 	| Spider 	| Webpage Description 	| Output 	|
|:--	|:--	|:--	|:--	|
| 1 	| hkjc-race 	| Racing results and dividend 	| race_YYYYMMDDhhmm.csv<br>dividend_YYYYMMDDhhmm.csv 	|
| 2 	| hkjc-sectional 	| Sectional times from racing results 	| sectional_YYYYMMDDhhmm.csv 	|
| 3 	| hkjc-horse 	| Horse information 	| horseform_YYYYMMDDhhmm.csv<br>horseinfo_YYYYMMDDhhmm.csv 	|

The spiders are run using the following command (in command line or powershell terminals):
```
scrapy crawl hkjc-race --logfile raceform_rerun.text
```
where `hkjc-race` can be subsituted for any of the above spiders. 

<b>Note:</b> scrapy-splash docker container has to be running and assumes default setting with `SPLASH_URL = 'http://localhost:8050'`, check access via browser to validate

The first data scraping pass was run on the racing results page (`hkjc-race`) using manually collected dates of the past 4 racing seasons i.e. 16/17 to 19/20 stored in `hkjc_racing_fixtures.csv`. This method was used to improve scraping efficiency instead of crawling through all dates for the past 4 years. Following this the `hkjc-sectional` spider was run on the same fixture set. 

The scraped racing results stored in the output file `race_YYYYMMDDhhmm.csv` contains a URL to horse information, and was used by the horse information crawler `hkjc-horse`. 

A second data scraping pass of the horse racing results and sectional times was run on the difference between dates of the results from the first pass and `horseform_YYYYMMDDhhmm.csv` output. 

### 1.2 PDF Parsing
On the horse information page active horses (not retired) contained a URL to prior overseas races which were stored as a PDF file. 
Tabula library was used to parse the horse racing results stored in the tables of the PDF into a csv file. The `overseas_pdfparser.py` takes the oversea URLs and generates the `hkjc-overseas.csv` output file. 
```
python3 overseas_pdfparser.py
```
However, as the PDF parsing is unstable with some columns moved out of position or values being associated to wrong columns due to spacing issues, the following code groups outputs by number of columns and generates unformatted CSV files. 

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import scrapy
import tabula

The `overseas_rerun.csv` file contains the overseas URLs which were not successfully parsed by `overseas_pdfparser.py`

In [193]:
df_rerun = pd.read_csv("./hkjc/overseas_rerun.csv", header = None)
pdf_urls = list(df_rerun[df_rerun.columns[0]])

In [197]:
df_d = {}

for i, pdf in enumerate(pdf_urls):
    pdf_url = 'https://racing.hkjc.com' + pdf
    df_pdf = tabula.read_pdf(pdf_url, stream=True)
    df_pdf = df_pdf.iloc[1:df_pdf.index[df_pdf[df_pdf.columns[0]]=="RACE RECORD SUMMARY"][0],:]
    df_pdf['overseas_url'] = pdf
    
    if len(df_pdf.columns) in df_d:
        df_d[len(df_pdf.columns)] = pd.concat([df_d[len(df_pdf.columns)], df_pdf])
    else:
        df_d[len(df_pdf.columns)] = df_pdf

Four csv output files were generated with number of columns ranging from 17 to 20. 

In [198]:
for k, v in df_d.items():
    v.to_csv('./overseas_rerun_'+str(k)+'.csv', index=False)

The four csv files were manually formatted so that values are associated to the correct columns. The following two functions parse this remaining data into the same format as the output from `overseas_pdfparser.py`

`rerun_mergecol` function merges columns of the same attribute with similar names which were split by Tabula due to spacing issues.

In [241]:
def rerun_mergecol(df):
    cols = ['date_venue','course_type', 'distance', 'going', 'race_name', 'group', 'position', 'draw', 
            'weight_carried', 'odds', 'gear', 'margin','jockey', 'race_time', 'race_value', 
            'prize', 'overseas_url']

    for col in cols:
        if len(df.filter(regex=col).columns) > 1:
            df[col] = df.filter(regex=col).apply(lambda x: ''.join(x.dropna().astype(str)).strip(), axis=1)
    
    return df

`rerun_parse` function applies the same logic from `overseas_pdfparser.py` so that the same output data can be merged

In [326]:
def rerun_parse(df):
    df_parse = df
    if len(df_parse.filter(regex='Unnamed').columns)>1:
        df_parse.columns = ['date_venue','course_type', 'distance', 'going', 'race_name', 'group', 'position', 'draw', 
                            'weight_carried', 'odds', 'gear', 'margin','jockey', 'race_time', 'race_value', 
                            'prize', 'overseas_url']
    
    agg_d = {'date_venue':lambda x: '|'.join(x.dropna()), 'course_type':'first', 'distance':'first', 'going':'first', 
             'race_name':lambda x: '|'.join(x.dropna()), 'group':'first', 'position':'first', 'draw':'first', 
             'weight_carried':'first', 'odds':'first', 'gear':'first', 'margin':'first', 'jockey':'first', 
             'race_time':lambda x: '|'.join(x.dropna()),'race_value':'first', 'prize':lambda x: '|'.join(x.dropna()),
             'overseas_url': 'first'}
    
    df_parse = df_parse.groupby(df_parse.position.notnull().cumsum().rename(None)).agg(agg_d)
    df_parse['date'] = df_parse['date_venue'].apply(lambda x: x[re.search(r'\|',x).span()[1]:].strip())
    df_parse['venue'] = df_parse['date_venue'].apply(lambda x: x[4:re.search(r'\|',x).span()[0]].strip())
    df_parse['prize_hkd'] = df_parse['prize'].apply(lambda x: int(x[re.search(r'\(HKD.*?\)',x).span()[0]+5: \
                                                            re.search(r'\((HKD.*?)\)', x).span()[1]-1].replace(',','')) \
                                                            if re.search(r'\(HKD.*?\)',x) else 0)
    df_parse['prize'] = df_parse['prize'].apply(lambda x: x[0:re.search(r'\d{1}\D{2}',x).span()[0]].strip())
    df_parse['finish_time'] = df_parse['race_time'].apply(lambda x: x[re.search(r'\(.*?\)',x).span()[0]+1: \
                                                                      re.search(r'\(.*?\)',x).span()[1]-1])
    df_parse['race_time'] = df_parse['race_time'].apply(lambda x: x[0:re.search(r'\|',x).span()[0]].strip())
    df_parse['course_type'] = df_parse[['venue', 'course_type']].apply(lambda row: row[0][-2:] if str(row[1]) == 'nan' \
                                                                                               else row[1], axis=1)
    
    return df_parse[['date', 'venue', 'course_type', 'distance' , 'going', 'race_name', 'group', 'position', 'draw', 
                     'weight_carried', 'odds', 'gear','margin','jockey', 'race_time', 'finish_time', 'race_value', 
                     'prize', 'prize_hkd', 'overseas_url']]

The 4 manually formatted overseas rerun csvs were labelled with the suffix '\_{no_columns}f' and passed through the above functions.

In [330]:
df_parse_d = {}
rerun_csv = ['17f', '18f', '19f', '20f']
for csv in rerun_csv:
    print(csv)
    df_rerun = pd.read_csv('overseas_rerun_'+csv+'.csv')
    df_rerun = rerun_mergecol(df_rerun)
    df_parse_d[csv] = rerun_parse(df_rerun)

17f
18f
19f
20f


The outputs are merged together and finally merged with the prior output from `overseas_parsepdf.py` to form a merged output of all overseas race results `hkjc-overseas-merged.csv`

In [333]:
i = 0
for k, v in df_parse_d.items():
    if i == 0:
        df_rerun_merged = v
    else:
        df_rerun_merged = pd.concat([df_rerun_merged, v])
    i += 1

In [335]:
df_overseas_orig = pd.read_csv('./hkjc/hkjc-overseas.csv')
df_overseas_merged = pd.concat([df_overseas_orig, df_rerun_merged])

df_overseas_merged.to_csv('hkjc-overseas-merged.csv', index = False)